In [1]:
import numpy as np
from docplex.cp.model import *

In [7]:
def calc_obj(x,proc_t,return_max=True):
    #calc value of objective function
    if return_max:
        return max(np.array(np.transpose(x)).dot(np.array(proc_t)))
    return np.array(np.transpose(x)).dot(np.array(proc_t))

def solve(proc_t,nb_m):
    #params: processing times, number of machines 
    mdl = CpoModel()
    
    #add bin decision variables
    x=[]
    
    for job_id in range(proc_t.shape[1]):
        x_row=[]
        for i in range(nb_m):
            curr_x=mdl.integer_var(0, 1,'x_%i%i' %(job_id,i))
            x_row.append(curr_x)
        x.append(x_row)
    #add every job needs to be mapped to exactly one machine constraint
    for job_id in range(proc_t.shape[1]):
        mdl.add(mdl.sum([x[job_id][mach_id] for mach_id in range(nb_m)])==1)
    
   
    #add lexicographic ordering
  
    for i in range(nb_m):
        if job_id < nb_m-1:
            mdl.lexicographic(np.array(x)[:,job_id],np.array(x)[:,job_id+1])
   
        
    #calculate sum of all job durations for every machine
    obj_var=[mdl.sum(x[job_id][mach_id]*proc_t[mach_id][job_id] for job_id in range(proc_t.shape[1]))for mach_id in range(nb_m)]
   
    #minimize the max duration of all machines
    mdl.minimize(mdl.max(obj_var))
    import time
    start = time.time()
    msol=mdl.solve()
    duration = time.time() - start
    sol=[]
    sol_row=[]

    for i,el in enumerate(msol.solution.var_solutions_list):
        sol_row.append(el.value)
        if((i+1)%nb_m==0):
            sol.append(sol_row)
            sol_row=[]
   
        
    return np.array(sol),msol.solution.objective_values, duration

In [24]:
def simulation(pt_mean,pt_std,nb_t_range,nb_m_range,nb_data,pt_int=False,return_dur=False):
    # init parameters
    # pt_mean: mean of processtime (duration of task on every machine)
    # pt_std:std of processtime 
    # number of task range f.e. [2,10] means uniform distribution between 2 and 10 tasks
    # number of machine range f.e. [2,10] means uniform distribution between 2 and 10 machines
    # nb_data = number of datasamples which should get generated
    # pt_int = (boolean) if process time is int or float (default: False)
    # return_dur = process time get's return (for statistics, default:False)
    x=[]
    y=[]
    if return_dur:
        dur=[]
    for i in range(nb_data):
        
        nb_m=np.random.randint(nb_m_range[0],nb_m_range[1])
        nb_t=np.random.randint(nb_t_range[0],nb_t_range[1])
        
        print('it:',i,'\t#m:',nb_m,'\t#t:',nb_t)
        if pt_int:
            proc_t=np.random.normal(loc=pt_mean,scale=pt_std,size=(nb_m,nb_t))
            proc_t=proc_t.astype(np.int64)
        else:
            proc_t=np.random.normal(loc=pt_mean,scale=pt_std,size=(nb_m,nb_t))
        #proc_t=np.array([[1,2,3,4,5],[2,1,3,5,4]])
        #nb_m=2       
        x.append(proc_t)
        
        sol=solve(proc_t,proc_t.shape[0])
        y.append(sol)
        if return_dur:
            dur.append(sol[-1])
    if return_dur:
        return x,y,dur
    return x,y

x,y=simulation(30,7,[3,17],[2,5],5000,True)
    

it: 0 	#m: 3 	#t: 7
it: 1 	#m: 2 	#t: 7
it: 2 	#m: 4 	#t: 11
it: 3 	#m: 2 	#t: 14
it: 4 	#m: 2 	#t: 16
it: 5 	#m: 2 	#t: 10
it: 6 	#m: 4 	#t: 12
it: 7 	#m: 2 	#t: 8
it: 8 	#m: 4 	#t: 5
it: 9 	#m: 2 	#t: 13
it: 10 	#m: 4 	#t: 5
it: 11 	#m: 4 	#t: 5
it: 12 	#m: 4 	#t: 15
it: 13 	#m: 4 	#t: 9
it: 14 	#m: 2 	#t: 9
it: 15 	#m: 2 	#t: 14
it: 16 	#m: 4 	#t: 8
it: 17 	#m: 2 	#t: 9
it: 18 	#m: 4 	#t: 11
it: 19 	#m: 2 	#t: 9
it: 20 	#m: 2 	#t: 4
it: 21 	#m: 4 	#t: 10
it: 22 	#m: 2 	#t: 13
it: 23 	#m: 4 	#t: 11
it: 24 	#m: 4 	#t: 3
it: 25 	#m: 3 	#t: 6
it: 26 	#m: 3 	#t: 14
it: 27 	#m: 2 	#t: 14
it: 28 	#m: 3 	#t: 6
it: 29 	#m: 3 	#t: 7
it: 30 	#m: 3 	#t: 10
it: 31 	#m: 4 	#t: 10
it: 32 	#m: 2 	#t: 11
it: 33 	#m: 2 	#t: 4
it: 34 	#m: 4 	#t: 9
it: 35 	#m: 4 	#t: 11
it: 36 	#m: 3 	#t: 11
it: 37 	#m: 2 	#t: 16
it: 38 	#m: 4 	#t: 3
it: 39 	#m: 3 	#t: 6
it: 40 	#m: 2 	#t: 6
it: 41 	#m: 4 	#t: 10
it: 42 	#m: 4 	#t: 8
it: 43 	#m: 4 	#t: 4
it: 44 	#m: 3 	#t: 12
it: 45 	#m: 4 	#t: 10
it: 46 	#m: 3 	#t: 3


it: 369 	#m: 2 	#t: 15
it: 370 	#m: 4 	#t: 7
it: 371 	#m: 2 	#t: 14
it: 372 	#m: 3 	#t: 3
it: 373 	#m: 4 	#t: 6
it: 374 	#m: 2 	#t: 12
it: 375 	#m: 2 	#t: 5
it: 376 	#m: 4 	#t: 5
it: 377 	#m: 4 	#t: 15
it: 378 	#m: 2 	#t: 14
it: 379 	#m: 4 	#t: 7
it: 380 	#m: 3 	#t: 16
it: 381 	#m: 2 	#t: 7
it: 382 	#m: 4 	#t: 8
it: 383 	#m: 3 	#t: 12
it: 384 	#m: 2 	#t: 5
it: 385 	#m: 4 	#t: 15
it: 386 	#m: 4 	#t: 3
it: 387 	#m: 3 	#t: 11
it: 388 	#m: 2 	#t: 6
it: 389 	#m: 4 	#t: 3
it: 390 	#m: 4 	#t: 5
it: 391 	#m: 4 	#t: 6
it: 392 	#m: 3 	#t: 5
it: 393 	#m: 2 	#t: 6
it: 394 	#m: 4 	#t: 6
it: 395 	#m: 4 	#t: 11
it: 396 	#m: 2 	#t: 13
it: 397 	#m: 4 	#t: 16
it: 398 	#m: 2 	#t: 10
it: 399 	#m: 2 	#t: 4
it: 400 	#m: 3 	#t: 5
it: 401 	#m: 2 	#t: 11
it: 402 	#m: 3 	#t: 11
it: 403 	#m: 3 	#t: 5
it: 404 	#m: 2 	#t: 14
it: 405 	#m: 3 	#t: 13
it: 406 	#m: 4 	#t: 6
it: 407 	#m: 4 	#t: 7
it: 408 	#m: 2 	#t: 13
it: 409 	#m: 3 	#t: 16
it: 410 	#m: 2 	#t: 7
it: 411 	#m: 3 	#t: 7
it: 412 	#m: 2 	#t: 12
it: 413 	#m:

it: 735 	#m: 2 	#t: 11
it: 736 	#m: 3 	#t: 4
it: 737 	#m: 4 	#t: 12
it: 738 	#m: 4 	#t: 9
it: 739 	#m: 2 	#t: 4
it: 740 	#m: 2 	#t: 15
it: 741 	#m: 4 	#t: 13
it: 742 	#m: 3 	#t: 4
it: 743 	#m: 4 	#t: 3
it: 744 	#m: 4 	#t: 6
it: 745 	#m: 2 	#t: 13
it: 746 	#m: 4 	#t: 15
it: 747 	#m: 4 	#t: 11
it: 748 	#m: 3 	#t: 5
it: 749 	#m: 3 	#t: 6
it: 750 	#m: 4 	#t: 6
it: 751 	#m: 2 	#t: 14
it: 752 	#m: 2 	#t: 16
it: 753 	#m: 2 	#t: 15
it: 754 	#m: 4 	#t: 3
it: 755 	#m: 2 	#t: 13
it: 756 	#m: 4 	#t: 12
it: 757 	#m: 4 	#t: 16
it: 758 	#m: 3 	#t: 11
it: 759 	#m: 4 	#t: 7
it: 760 	#m: 4 	#t: 7
it: 761 	#m: 2 	#t: 6
it: 762 	#m: 3 	#t: 9
it: 763 	#m: 2 	#t: 8
it: 764 	#m: 2 	#t: 15
it: 765 	#m: 3 	#t: 10
it: 766 	#m: 4 	#t: 13
it: 767 	#m: 2 	#t: 8
it: 768 	#m: 3 	#t: 11
it: 769 	#m: 2 	#t: 14
it: 770 	#m: 2 	#t: 3
it: 771 	#m: 3 	#t: 3
it: 772 	#m: 4 	#t: 11
it: 773 	#m: 4 	#t: 13
it: 774 	#m: 4 	#t: 13
it: 775 	#m: 4 	#t: 3
it: 776 	#m: 2 	#t: 16
it: 777 	#m: 2 	#t: 10
it: 778 	#m: 4 	#t: 3
it: 779 

it: 1096 	#m: 3 	#t: 7
it: 1097 	#m: 2 	#t: 13
it: 1098 	#m: 2 	#t: 5
it: 1099 	#m: 4 	#t: 3
it: 1100 	#m: 2 	#t: 8
it: 1101 	#m: 2 	#t: 14
it: 1102 	#m: 4 	#t: 6
it: 1103 	#m: 2 	#t: 11
it: 1104 	#m: 2 	#t: 8
it: 1105 	#m: 4 	#t: 3
it: 1106 	#m: 3 	#t: 7
it: 1107 	#m: 4 	#t: 16
it: 1108 	#m: 3 	#t: 5
it: 1109 	#m: 2 	#t: 15
it: 1110 	#m: 3 	#t: 7
it: 1111 	#m: 2 	#t: 11
it: 1112 	#m: 4 	#t: 13
it: 1113 	#m: 2 	#t: 6
it: 1114 	#m: 3 	#t: 5
it: 1115 	#m: 4 	#t: 13
it: 1116 	#m: 3 	#t: 13
it: 1117 	#m: 2 	#t: 4
it: 1118 	#m: 4 	#t: 5
it: 1119 	#m: 2 	#t: 8
it: 1120 	#m: 2 	#t: 10
it: 1121 	#m: 2 	#t: 3
it: 1122 	#m: 3 	#t: 8
it: 1123 	#m: 2 	#t: 7
it: 1124 	#m: 3 	#t: 9
it: 1125 	#m: 2 	#t: 8
it: 1126 	#m: 2 	#t: 6
it: 1127 	#m: 2 	#t: 16
it: 1128 	#m: 2 	#t: 4
it: 1129 	#m: 3 	#t: 5
it: 1130 	#m: 4 	#t: 13
it: 1131 	#m: 3 	#t: 3
it: 1132 	#m: 2 	#t: 3
it: 1133 	#m: 4 	#t: 4
it: 1134 	#m: 3 	#t: 6
it: 1135 	#m: 4 	#t: 3
it: 1136 	#m: 2 	#t: 3
it: 1137 	#m: 4 	#t: 10
it: 1138 	#m: 3 	#t: 

it: 1448 	#m: 2 	#t: 11
it: 1449 	#m: 3 	#t: 7
it: 1450 	#m: 3 	#t: 12
it: 1451 	#m: 4 	#t: 8
it: 1452 	#m: 3 	#t: 16
it: 1453 	#m: 2 	#t: 6
it: 1454 	#m: 2 	#t: 15
it: 1455 	#m: 3 	#t: 3
it: 1456 	#m: 4 	#t: 10
it: 1457 	#m: 2 	#t: 8
it: 1458 	#m: 3 	#t: 6
it: 1459 	#m: 3 	#t: 3
it: 1460 	#m: 4 	#t: 7
it: 1461 	#m: 3 	#t: 10
it: 1462 	#m: 2 	#t: 3
it: 1463 	#m: 4 	#t: 14
it: 1464 	#m: 2 	#t: 4
it: 1465 	#m: 3 	#t: 7
it: 1466 	#m: 3 	#t: 16
it: 1467 	#m: 4 	#t: 10
it: 1468 	#m: 3 	#t: 3
it: 1469 	#m: 2 	#t: 12
it: 1470 	#m: 3 	#t: 3
it: 1471 	#m: 4 	#t: 14
it: 1472 	#m: 3 	#t: 6
it: 1473 	#m: 2 	#t: 13
it: 1474 	#m: 4 	#t: 16
it: 1475 	#m: 3 	#t: 7
it: 1476 	#m: 2 	#t: 3
it: 1477 	#m: 3 	#t: 12
it: 1478 	#m: 2 	#t: 11
it: 1479 	#m: 3 	#t: 15
it: 1480 	#m: 4 	#t: 6
it: 1481 	#m: 3 	#t: 8
it: 1482 	#m: 2 	#t: 16
it: 1483 	#m: 3 	#t: 12
it: 1484 	#m: 3 	#t: 13
it: 1485 	#m: 3 	#t: 8
it: 1486 	#m: 4 	#t: 9
it: 1487 	#m: 4 	#t: 8
it: 1488 	#m: 3 	#t: 13
it: 1489 	#m: 2 	#t: 8
it: 1490 	#m: 

...

it: 3552 	#m: 3 	#t: 13
it: 3553 	#m: 2 	#t: 12
it: 3554 	#m: 3 	#t: 4
it: 3555 	#m: 4 	#t: 10
it: 3556 	#m: 2 	#t: 6
it: 3557 	#m: 4 	#t: 16
it: 3558 	#m: 3 	#t: 4
it: 3559 	#m: 4 	#t: 4
it: 3560 	#m: 4 	#t: 16
it: 3561 	#m: 3 	#t: 11
it: 3562 	#m: 4 	#t: 9
it: 3563 	#m: 2 	#t: 12
it: 3564 	#m: 3 	#t: 16
it: 3565 	#m: 2 	#t: 12
it: 3566 	#m: 2 	#t: 11
it: 3567 	#m: 3 	#t: 15
it: 3568 	#m: 2 	#t: 4
it: 3569 	#m: 2 	#t: 4
it: 3570 	#m: 4 	#t: 6
it: 3571 	#m: 3 	#t: 11
it: 3572 	#m: 4 	#t: 9
it: 3573 	#m: 3 	#t: 12
it: 3574 	#m: 2 	#t: 14
it: 3575 	#m: 2 	#t: 3
it: 3576 	#m: 4 	#t: 13
it: 3577 	#m: 2 	#t: 11
it: 3578 	#m: 2 	#t: 15
it: 3579 	#m: 4 	#t: 14
it: 3580 	#m: 3 	#t: 10
it: 3581 	#m: 2 	#t: 9
it: 3582 	#m: 4 	#t: 3
it: 3583 	#m: 3 	#t: 11
it: 3584 	#m: 3 	#t: 5
it: 3585 	#m: 4 	#t: 11
it: 3586 	#m: 3 	#t: 11
it: 3587 	#m: 3 	#t: 14
it: 3588 	#m: 2 	#t: 8
it: 3589 	#m: 4 	#t: 7
it: 3590 	#m: 4 	#t: 13
it: 3591 	#m: 3 	#t: 12
it: 3592 	#m: 3 	#t: 10
it: 3593 	#m: 3 	#t: 13
it: 359

it: 3902 	#m: 4 	#t: 6
it: 3903 	#m: 2 	#t: 9
it: 3904 	#m: 2 	#t: 6
it: 3905 	#m: 2 	#t: 3
it: 3906 	#m: 2 	#t: 13
it: 3907 	#m: 3 	#t: 9
it: 3908 	#m: 4 	#t: 6
it: 3909 	#m: 4 	#t: 8
it: 3910 	#m: 2 	#t: 12
it: 3911 	#m: 2 	#t: 6
it: 3912 	#m: 2 	#t: 3
it: 3913 	#m: 3 	#t: 7
it: 3914 	#m: 4 	#t: 9
it: 3915 	#m: 2 	#t: 14
it: 3916 	#m: 3 	#t: 12
it: 3917 	#m: 2 	#t: 4
it: 3918 	#m: 3 	#t: 11
it: 3919 	#m: 3 	#t: 10
it: 3920 	#m: 4 	#t: 7
it: 3921 	#m: 4 	#t: 13
it: 3922 	#m: 4 	#t: 9
it: 3923 	#m: 2 	#t: 4
it: 3924 	#m: 3 	#t: 3
it: 3925 	#m: 2 	#t: 7
it: 3926 	#m: 3 	#t: 16
it: 3927 	#m: 4 	#t: 14
it: 3928 	#m: 3 	#t: 15
it: 3929 	#m: 2 	#t: 12
it: 3930 	#m: 3 	#t: 7
it: 3931 	#m: 3 	#t: 7
it: 3932 	#m: 4 	#t: 5
it: 3933 	#m: 3 	#t: 4
it: 3934 	#m: 2 	#t: 14
it: 3935 	#m: 2 	#t: 3
it: 3936 	#m: 2 	#t: 9
it: 3937 	#m: 4 	#t: 5
it: 3938 	#m: 2 	#t: 7
it: 3939 	#m: 2 	#t: 4
it: 3940 	#m: 2 	#t: 11
it: 3941 	#m: 3 	#t: 13
it: 3942 	#m: 2 	#t: 8
it: 3943 	#m: 3 	#t: 7
it: 3944 	#m: 4 	#t:

it: 4251 	#m: 2 	#t: 16
it: 4252 	#m: 3 	#t: 16
it: 4253 	#m: 2 	#t: 12
it: 4254 	#m: 2 	#t: 11
it: 4255 	#m: 2 	#t: 16
it: 4256 	#m: 2 	#t: 14
it: 4257 	#m: 4 	#t: 4
it: 4258 	#m: 4 	#t: 4
it: 4259 	#m: 4 	#t: 14
it: 4260 	#m: 4 	#t: 13
it: 4261 	#m: 2 	#t: 13
it: 4262 	#m: 3 	#t: 7
it: 4263 	#m: 2 	#t: 8
it: 4264 	#m: 4 	#t: 16
it: 4265 	#m: 4 	#t: 7
it: 4266 	#m: 3 	#t: 9
it: 4267 	#m: 2 	#t: 11
it: 4268 	#m: 2 	#t: 10
it: 4269 	#m: 2 	#t: 4
it: 4270 	#m: 3 	#t: 9
it: 4271 	#m: 3 	#t: 14
it: 4272 	#m: 4 	#t: 3
it: 4273 	#m: 4 	#t: 15
it: 4274 	#m: 2 	#t: 7
it: 4275 	#m: 2 	#t: 12
it: 4276 	#m: 3 	#t: 8
it: 4277 	#m: 3 	#t: 16
it: 4278 	#m: 3 	#t: 14
it: 4279 	#m: 3 	#t: 4
it: 4280 	#m: 4 	#t: 8
it: 4281 	#m: 4 	#t: 4
it: 4282 	#m: 3 	#t: 15
it: 4283 	#m: 3 	#t: 3
it: 4284 	#m: 3 	#t: 11
it: 4285 	#m: 3 	#t: 16
it: 4286 	#m: 4 	#t: 11
it: 4287 	#m: 2 	#t: 10
it: 4288 	#m: 3 	#t: 12
it: 4289 	#m: 4 	#t: 14
it: 4290 	#m: 4 	#t: 10
it: 4291 	#m: 3 	#t: 16
it: 4292 	#m: 2 	#t: 15
it: 429

it: 4601 	#m: 3 	#t: 6
it: 4602 	#m: 3 	#t: 16
it: 4603 	#m: 4 	#t: 8
it: 4604 	#m: 2 	#t: 9
it: 4605 	#m: 4 	#t: 11
it: 4606 	#m: 4 	#t: 11
it: 4607 	#m: 4 	#t: 10
it: 4608 	#m: 2 	#t: 14
it: 4609 	#m: 3 	#t: 14
it: 4610 	#m: 4 	#t: 14
it: 4611 	#m: 2 	#t: 8
it: 4612 	#m: 2 	#t: 6
it: 4613 	#m: 3 	#t: 14
it: 4614 	#m: 4 	#t: 14
it: 4615 	#m: 2 	#t: 4
it: 4616 	#m: 4 	#t: 15
it: 4617 	#m: 4 	#t: 5
it: 4618 	#m: 2 	#t: 13
it: 4619 	#m: 3 	#t: 16
it: 4620 	#m: 2 	#t: 8
it: 4621 	#m: 4 	#t: 16
it: 4622 	#m: 4 	#t: 3
it: 4623 	#m: 2 	#t: 11
it: 4624 	#m: 4 	#t: 12
it: 4625 	#m: 2 	#t: 12
it: 4626 	#m: 3 	#t: 13
it: 4627 	#m: 3 	#t: 12
it: 4628 	#m: 2 	#t: 10
it: 4629 	#m: 4 	#t: 3
it: 4630 	#m: 4 	#t: 5
it: 4631 	#m: 2 	#t: 8
it: 4632 	#m: 2 	#t: 16
it: 4633 	#m: 2 	#t: 3
it: 4634 	#m: 2 	#t: 14
it: 4635 	#m: 4 	#t: 5
it: 4636 	#m: 3 	#t: 9
it: 4637 	#m: 3 	#t: 5
it: 4638 	#m: 4 	#t: 7
it: 4639 	#m: 4 	#t: 3
it: 4640 	#m: 3 	#t: 10
it: 4641 	#m: 4 	#t: 11
it: 4642 	#m: 2 	#t: 14
it: 4643 	

it: 4950 	#m: 4 	#t: 8
it: 4951 	#m: 2 	#t: 8
it: 4952 	#m: 2 	#t: 15
it: 4953 	#m: 4 	#t: 6
it: 4954 	#m: 2 	#t: 8
it: 4955 	#m: 3 	#t: 11
it: 4956 	#m: 4 	#t: 9
it: 4957 	#m: 3 	#t: 9
it: 4958 	#m: 4 	#t: 10
it: 4959 	#m: 2 	#t: 4
it: 4960 	#m: 2 	#t: 4
it: 4961 	#m: 4 	#t: 5
it: 4962 	#m: 4 	#t: 14
it: 4963 	#m: 2 	#t: 4
it: 4964 	#m: 4 	#t: 15
it: 4965 	#m: 4 	#t: 15
it: 4966 	#m: 3 	#t: 4
it: 4967 	#m: 3 	#t: 8
it: 4968 	#m: 3 	#t: 14
it: 4969 	#m: 2 	#t: 15
it: 4970 	#m: 4 	#t: 6
it: 4971 	#m: 4 	#t: 3
it: 4972 	#m: 2 	#t: 11
it: 4973 	#m: 4 	#t: 4
it: 4974 	#m: 3 	#t: 11
it: 4975 	#m: 2 	#t: 10
it: 4976 	#m: 3 	#t: 13
it: 4977 	#m: 2 	#t: 8
it: 4978 	#m: 3 	#t: 3
it: 4979 	#m: 3 	#t: 9
it: 4980 	#m: 2 	#t: 5
it: 4981 	#m: 2 	#t: 8
it: 4982 	#m: 4 	#t: 9
it: 4983 	#m: 2 	#t: 15
it: 4984 	#m: 3 	#t: 13
it: 4985 	#m: 4 	#t: 14
it: 4986 	#m: 3 	#t: 10
it: 4987 	#m: 3 	#t: 10
it: 4988 	#m: 3 	#t: 13
it: 4989 	#m: 3 	#t: 16
it: 4990 	#m: 3 	#t: 3
it: 4991 	#m: 4 	#t: 11
it: 4992 	#m: 

In [4]:

x

[array([[19, 31, 29, 33, 31, 39, 23, 23, 34, 22, 34, 10, 29, 25, 34],
        [22, 33, 31, 32, 25, 35, 31, 31, 25, 32, 16, 22, 23, 23, 30],
        [19, 33, 35, 28, 23, 31, 28, 32, 16, 37, 26, 33, 33, 29, 31]],
       dtype=int64)]

In [5]:
print(y)
print(np.transpose(y[0][0]))

[(array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]]), (122,), 0.4901111125946045)]
[[0 1 1 0 0 0 0 1 0 1 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 1 0 1 1 1]
 [1 0 0 1 0 1 1 0 1 0 0 0 0 0 0]]


In [29]:
#write output to .csv
def write_output(arr,filename):
    #takes filename and np array of data and write it to .csv
    with open(filename, 'w') as of:

        for row in np.array(arr):
            line="["
            for i,elem in enumerate( row):
                line=line+'['
                for j,nb in enumerate( elem):
                    if j!=len(elem)-1:

                        line=line+str(nb)+','


                if i!=len(row)-1:
                    line=line+str(nb)+'],'

            of.write(line+str(nb)+']]\n')       
        

In [31]:
write_output(x,'input_pt2.csv')
write_output(np.array(y)[:,0],'output2.csv')

In [19]:
np.array(y)[:,0]

array([array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0]]),
       array([[0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0]]),
       array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0]]),
       array([[0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1